In [0]:
#imports and display settings
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)

In [0]:
# 1.Load datasets 
# companies.txt may be tab/pipe/comma separated; adjust sep parameter if required.
companies = pd.read_csv('/Volumes/workspace/default/usecase/companies.txt', sep='\t', encoding='ISO-8859-1', low_memory=False)


investments = pd.read_csv('/Volumes/workspace/default/usecase/InvestmentData.csv', encoding='ISO-8859-1',low_memory=False)
mapping = pd.read_csv('/Volumes/workspace/default/usecase/mapping.csv',encoding='ISO-8859-1', low_memory=False)

companies.head(2)
investments.head(2)
mapping.head(2)


,category_list,Automotive & Sports,Blanks,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising"
0,NaN,0,1,0,0,0,0,0,0,0
1,3D,0,0,0,0,0,1,0,0,0


In [0]:
#2. Normalize column names
companies.columns = companies.columns.str.strip().str.lower()
investments.columns = investments.columns.str.strip().str.lower()
mapping.columns = mapping.columns.str.strip().str.lower()

print('companies cols:', companies.columns.tolist()[:20])
print('investments cols:', investments.columns.tolist()[:20])
print('mapping cols:', mapping.columns.tolist()[:40])


companies cols: ['permalink', 'name', 'homepage_url', 'category_list', 'status', 'country_code', 'state_code', 'region', 'city', 'founded_at']
investments cols: ['company_permalink', 'funding_round_permalink', 'funding_round_type', 'funding_round_code', 'funded_at', 'raised_amount_usd']
mapping cols: ['category_list', 'automotive & sports', 'blanks', 'cleantech / semiconductors', 'entertainment', 'health', 'manufacturing', 'news, search and messaging', 'others', 'social, finance, analytics, advertising']


In [0]:
# 3.Merge datasets
# Ensure company_permalink/permalink matching format: sometimes investments company_permalink includes leading '/'
investments['company_permalink'] = investments['company_permalink'].astype(str).str.lower().str.strip()
companies['permalink'] = companies['permalink'].astype(str).str.lower().str.strip()

master = pd.merge(investments, companies, how='inner',left_on='company_permalink', right_on='permalink', suffixes=('_inv','_comp'))

print('master shape:', master.shape)
master[['company_permalink','permalink','funding_round_type','raised_amount_usd','country_code','category_list']].head(6)

master shape: (114942, 16)


,company_permalink,permalink,funding_round_type,raised_amount_usd,country_code,category_list
0,/organization/-fame,/organization/-fame,venture,10000000.0,IND,Media
1,/organization/-qounter,/organization/-qounter,venture,NaN,USA,Application Platforms|Real Time|Social Network...
2,/organization/-qounter,/organization/-qounter,seed,700000.0,USA,Application Platforms|Real Time|Social Network...
3,/organization/-the-one-of-them-inc-,/organization/-the-one-of-them-inc-,venture,3406878.0,NaN,Apps|Games|Mobile
4,/organization/0-6-com,/organization/0-6-com,venture,2000000.0,CHN,Curated Web
5,/organization/004-technologies,/organization/004-technologies,venture,NaN,USA,Software


In [0]:
#4.Extract primary sector from category_list
def extract_primary(cat):
    if pd.isna(cat): return np.nan
    # some files use '|' separator, some use ','; handle both
    if '|' in cat:
        return cat.split('|')[0].strip().lower()
    else:
        return str(cat).split(',')[0].strip().lower()

master['primary_sector'] = master['category_list'].apply(extract_primary)
master['primary_sector'] = master['primary_sector'].replace('', np.nan)
master[['category_list','primary_sector']].head(10)

,category_list,primary_sector
0,Media,media
1,Application Platforms|Real Time|Social Network...,application platforms
2,Application Platforms|Real Time|Social Network...,application platforms
3,Apps|Games|Mobile,apps
4,Curated Web,curated web
5,Software,software
6,Games,games
7,Biotechnology,biotechnology
8,Biotechnology,biotechnology
9,Analytics,analytics


In [0]:
# 5.Clean and reshape mapping.csv
# mapping usually has a 'category_list' column and many columns representing main sectors with 0/1 flags.
mapping.columns = mapping.columns.str.strip().str.lower()
if 'category_list' not in mapping.columns:
    # try first column name as category column
    mapping = mapping.rename(columns={mapping.columns[0]:'category_list'})

mapping['category_list'] = mapping['category_list'].astype(str).str.strip().str.lower()

# Melt to long format
value_cols = [c for c in mapping.columns if c!='category_list']
melt = mapping.melt(id_vars=['category_list'], value_vars=value_cols, var_name='main_sector', value_name='flag')

# Keep only flagged rows (flag likely 1 or True)
melt = melt[melt['flag'].astype(str).isin(['1','1.0','True','true'])]
melt = melt[['category_list','main_sector']].drop_duplicates().reset_index(drop=True)

print('mapping (long) sample:')
melt.head(10)


mapping (long) sample:


,category_list,main_sector
0,adventure travel,automotive & sports
1,aerospace,automotive & sports
2,auto,automotive & sports
3,automated kiosk,automotive & sports
4,automotive,automotive & sports
5,bicycles,automotive & sports
6,boating industry,automotive & sports
7,cad,automotive & sports
8,cars,automotive & sports
9,design,automotive & sports


In [0]:
# 6. Merge master with mapping on primary sector
# Both sides lowercase already
melt['category_list'] = melt['category_list'].str.lower().str.strip()
master['primary_sector'] = master['primary_sector'].str.lower().str.strip()

master = master.merge(melt, how='left', left_on='primary_sector', right_on='category_list')
print('After mapping merge, sample:')
master[['primary_sector','main_sector']].drop_duplicates().head(20)


After mapping merge, sample:


,primary_sector,main_sector
0,media,entertainment
1,application platforms,"news, search and messaging"
3,apps,"news, search and messaging"
4,curated web,"news, search and messaging"
5,software,others
6,games,entertainment
7,biotechnology,cleantech / semiconductors
9,analytics,NaN
13,mobile,"news, search and messaging"
16,e-commerce,others


In [0]:
# 7. Validate mapping results and inspect unmapped primary sectors
unmapped = master[master['main_sector'].isnull()]['primary_sector'].unique()
print('Number of unmapped primary_sector values:', len(unmapped))
print(unmapped[:50])
# If many unmapped, consider inspecting mapping file or filling manually

Number of unmapped primary_sector values: 94
['analytics' nan 'finance' 'financial services' 'project management'
 'business analytics' 'career management' 'nanotechnology'
 'finance technology' 'contact management' 'big data analytics'
 'self development' 'cloud management' 'digital signage' 'cause marketing'
 'personalization' 'innovation management' 'risk management'
 'document management' 'journalism' 'investment management' 'cannabis'
 'energy management' 'waste management' 'fleet management'
 'property management' 'alternative medicine' 'real estate investors'
 'english-speaking' 'intellectual asset management' 'event management'
 'educational games' 'identity management' 'lead management'
 'it management' 'navigation' 'deep information technology' 'toys'
 'professional services' 'generation y-z' 'spas'
 'digital rights management' 'enterprise hardware'
 'social media advertising' 'personal finance' 'darknet'
 'knowledge management' 'china internet' 'medical professionals'
 'natu

In [0]:
# 8. Choose funding type by average raised amount (range 5M to 15M recommended)
funding_type_avg = (master.groupby('funding_round_type')['raised_amount_usd']
                    .mean().reset_index().sort_values(by='raised_amount_usd', ascending=False))
funding_type_avg['raised_amount_usd_million'] = funding_type_avg['raised_amount_usd']/1e6
funding_type_avg


,funding_round_type,raised_amount_usd,raised_amount_usd_million
6,post_ipo_debt,1.687046e+08,168.704572
7,post_ipo_equity,8.218249e+07,82.182494
10,secondary_market,7.964963e+07,79.649630
8,private_equity,7.330859e+07,73.308593
12,undisclosed,1.924237e+07,19.242370
2,debt_financing,1.704353e+07,17.043526
13,venture,1.174895e+07,11.748949
4,grant,4.300576e+06,4.300576
1,convertible_note,1.453439e+06,1.453439
9,product_crowdfunding,1.363131e+06,1.363131


In [0]:
# Choose funding type 
# Common recommended: 'venture'
chosen_type = 'venture'
df = master[master['funding_round_type']==chosen_type].copy()
print('Records selected:', df.shape[0])
df[['funding_round_type','raised_amount_usd']].head(3)

Records selected: 55493


,funding_round_type,raised_amount_usd
0,venture,10000000.0
1,venture,NaN
3,venture,3406878.0


In [0]:
#9.Top countries by total funding (for chosen funding type)
country_total = (df.groupby('country_code')['raised_amount_usd']
                   .sum().reset_index().sort_values(by='raised_amount_usd', ascending=False))
top9 = country_total.head(9).copy()
top9['raised_amount_usd_million'] = top9['raised_amount_usd']/1e6
top9


,country_code,raised_amount_usd,raised_amount_usd_million
100,USA,4.225108e+11,422510.842796
16,CHN,3.983542e+10,39835.418773
31,GBR,2.024563e+10,20245.627416
41,IND,1.439186e+10,14391.858718
13,CAN,9.583332e+09,9583.332317
30,FRA,7.259537e+09,7259.536732
44,ISR,6.907515e+09,6907.514579
23,DEU,6.346960e+09,6346.959822
47,JPN,3.363677e+09,3363.676611


In [0]:
#10.Select top 3 countries
top3 = top9['country_code'].head(3).tolist()
print('Top 3 countries:', top3)
df_top3 = df[df['country_code'].isin(top3)].copy()
df_top3.shape

Top 3 countries: ['USA', 'CHN', 'GBR']


(42762, 19)

In [0]:
#11.Sector analysis within top3 countries
sector_analysis = (df_top3.groupby(['country_code','main_sector'])
                     .agg({'raised_amount_usd':'sum','company_permalink':'count'})
                     .rename(columns={'company_permalink':'investment_count'})
                     .reset_index()
                     .sort_values(by=['country_code','raised_amount_usd'], ascending=[True,False]))
sector_analysis['raised_amount_usd_million'] = sector_analysis['raised_amount_usd']/1e6
sector_analysis.head(20)

,country_code,main_sector,raised_amount_usd,investment_count,raised_amount_usd_million
6,CHN,others,9.265594e+09,613,9265.593610
7,CHN,"social, finance, analytics, advertising",7.262401e+09,303,7262.400734
5,CHN,"news, search and messaging",6.454233e+09,330,6454.233272
3,CHN,health,4.402226e+09,122,4402.226232
2,CHN,entertainment,3.486604e+09,261,3486.604384
4,CHN,manufacturing,3.424206e+09,151,3424.206239
1,CHN,cleantech / semiconductors,1.669178e+09,142,1669.178213
0,CHN,automotive & sports,1.066993e+09,37,1066.993443
9,GBR,cleantech / semiconductors,5.028729e+09,456,5028.729267
14,GBR,others,4.400423e+09,571,4400.423043


In [0]:
#12. Top sector per each top country
top_sector_per_country = sector_analysis.groupby('country_code').first().reset_index()
top_sector_per_country[['country_code','main_sector','raised_amount_usd_million']]


,country_code,main_sector,raised_amount_usd_million
0,CHN,others,9265.593610
1,GBR,cleantech / semiconductors,5028.729267
2,USA,cleantech / semiconductors,118764.441980


In [0]:
#13. For each country top sector, find top funded company
def get_top_company(country, sector):
    temp = df_top3[(df_top3['country_code']==country) & (df_top3['main_sector']==sector)]
    if temp.empty:
        return None
    row = temp.loc[temp['raised_amount_usd'].idxmax()]
    return pd.Series({'company_permalink': row['company_permalink'], 'raised_amount_usd': row['raised_amount_usd']})

results = []
for _, r in top_sector_per_country.iterrows():
    country = r['country_code']
    sector = r['main_sector']
    s = get_top_company(country, sector)
    if s is not None:
        s['country_code'] = country
        s['main_sector'] = sector
        results.append(s)
top_companies = pd.DataFrame(results)
top_companies


,company_permalink,raised_amount_usd,country_code,main_sector
0,/organization/koudai,3.500000e+08,CHN,others
1,/organization/immunocore,3.200000e+08,GBR,cleantech / semiconductors
2,/organization/freescale,1.760000e+10,USA,cleantech / semiconductors
